Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-with-automated-machine-learning-step.png)

# Azure Machine Learning Pipeline with AutoMLStep (Udacity Course 2)
This notebook demonstrates the use of AutoMLStep in Azure Machine Learning Pipeline.

## Introduction
In this example we showcase how you can use AzureML Dataset to load data for AutoML via AML Pipeline. 

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you have executed the [configuration](https://aka.ms/pl-config) before running this notebook.

In this notebook you will learn how to:
1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a `TabularDataset`.
4. Configure AutoML using `AutoMLConfig`.
5. Use AutoMLStep
6. Train the model using AmlCompute
7. Explore the results.
8. Test the best fitted model.

## Azure Machine Learning and Pipeline SDK-specific imports

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.60.0


## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

udacity_ws
udacity_rg
eastus
051560d6-9344-4907-a9c5-057add5cf030


## Create an Azure ML experiment
Let's create an experiment named "automlstep-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

*Udacity Note:* There is no need to create an Azure ML experiment, this needs to re-use the experiment that was already created


In [4]:
experiment_name = 'bm_job'
project_folder = './bm-job-automlstep-classification'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
bm_job,udacity_ws,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. In this tutorial, you get the default `AmlCompute` as your training compute resource.

**Udacity Note** There is no need to create a new compute target, it can re-use the previous cluster

In [6]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

amlcompute_cluster_name = "ops-compute"

try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.get_status()

Found existing cluster, use it.


{
  "errors": [],
  "creationTime": "2025-06-22T11:41:22.912846+00:00",
  "createdBy": {
    "userObjectId": "fe75403a-f3c0-4149-ba5e-7607c21d4f2b",
    "userTenantId": "a54ddfa5-3d29-437b-a420-b882e37e67e2",
    "userName": null
  },
  "modifiedTime": "2025-06-22T11:47:58.715305+00:00",
  "state": "Running",
  "vmSize": "Standard_D4s_v3"
}

## Data

**Udacity note:** Make sure the `key` is the same name as the dataset that is uploaded, and that the description matches. If it is hard to find or unknown, loop over the `ws.datasets.keys()` and `print()` them.
If it *isn't* found because it was deleted, it can be recreated with the link that has the CSV 

In [7]:
found = False
key = "bankmarketing"
description_text = "Bike Sharing DataSet for Udacity Course 2"

if key in ws.datasets.keys(): 
    found = True
    dataset = ws.datasets[key] 

if not found:
    # Create AML Dataset and register it into Workspace
    example_data = 'https://raw.githubusercontent.com/Azure/MachineLearningNotebooks/master/how-to-use-azureml/automated-machine-learning/forecasting-bike-share/bike-no.csv'
    dataset = Dataset.Tabular.from_delimited_files(example_data)        
    #Register Dataset in Workspace
    dataset = dataset.register(workspace=ws,
                               name=key,
                               description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000
mean,40.040212,257.335205,2.561730,962.174780,0.174780,0.076228,93.574243,-40.518680,3.615654,5166.859608
std,10.432313,257.331700,2.763646,187.646785,0.496503,1.572242,0.578636,4.623004,1.735748,72.208448
min,17.000000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.000000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.000000,179.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.000000,318.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.000000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


### Review the Dataset Result

You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only `j` records for all the steps in the TabularDataset, which makes it fast even against large datasets.

`TabularDataset` objects are composed of a list of transformation steps (optional).

In [8]:
dataset.take(5).to_pandas_dataframe()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


## Train
This creates a general AutoML settings object.
**Udacity notes:** These inputs must match what was used when training in the portal. `time_column_name` has to be `cnt` for example.

In [9]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 3
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="y",  
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

#### Create Pipeline and AutoMLStep

You can define outputs for the AutoMLStep using TrainingOutput.

In [10]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

Create an AutoMLStep.

In [11]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [12]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [13]:
pipeline_run = experiment.submit(pipeline)

Created step automl_module [f69b5cb3][300205cb-9b12-47ed-90a4-e0add3647ad1], (This step will run and generate new outputs)
Submitted PipelineRun 6277ee6c-48bc-4532-adc2-e95fd790b9c8
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/6277ee6c-48bc-4532-adc2-e95fd790b9c8?wsid=/subscriptions/051560d6-9344-4907-a9c5-057add5cf030/resourcegroups/udacity_rg/workspaces/udacity_ws&tid=a54ddfa5-3d29-437b-a420-b882e37e67e2


In [17]:
# from azureml.widgets import RunDetails
# RunDetails(pipeline_run).show()

for run in experiment.get_runs():
    print(f"Run ID: {run.id} | Status: {run.status}")

Run ID: 6277ee6c-48bc-4532-adc2-e95fd790b9c8 | Status: Completed


In [16]:
pipeline_run.wait_for_completion()

PipelineRunId: 6277ee6c-48bc-4532-adc2-e95fd790b9c8
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/6277ee6c-48bc-4532-adc2-e95fd790b9c8?wsid=/subscriptions/051560d6-9344-4907-a9c5-057add5cf030/resourcegroups/udacity_rg/workspaces/udacity_ws&tid=a54ddfa5-3d29-437b-a420-b882e37e67e2

PipelineRun Execution Summary
PipelineRun Status: Finished
{'runId': '6277ee6c-48bc-4532-adc2-e95fd790b9c8', 'status': 'Completed', 'startTimeUtc': '2025-06-22T13:23:33.954818Z', 'endTimeUtc': '2025-06-22T13:45:13.947654Z', 'services': {}, 'properties': {'azureml.runsource': 'azureml.PipelineRun', 'runSource': 'SDK', 'runType': 'SDK', 'azureml.parameters': '{}', 'azureml.continue_on_step_failure': 'False', 'azureml.continue_on_failed_optional_input': 'True', 'azureml.pipelineComponent': 'pipelinerun', 'azureml.pipelines.stages': '{"Initialization":null,"Execution":{"StartTime":"2025-06-22T13:23:34.2859994+00:00","EndTime":"2025-06-22T13:45:13.7865439+00:00","Status":"Finished"}}'}, 'inputDa

'Finished'

## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will examine the outputs by retrieve output data and running some tests.

In [19]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

Downloaded azureml/67f22d40-f07f-4ee3-bbbd-5f1ed9d27e69/metrics_data, 1 files out of an estimated total of 1


In [20]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

,67f22d40-f07f-4ee3-bbbd-5f1ed9d27e69_2,67f22d40-f07f-4ee3-bbbd-5f1ed9d27e69_0,67f22d40-f07f-4ee3-bbbd-5f1ed9d27e69_5,67f22d40-f07f-4ee3-bbbd-5f1ed9d27e69_1,67f22d40-f07f-4ee3-bbbd-5f1ed9d27e69_6,67f22d40-f07f-4ee3-bbbd-5f1ed9d27e69_33,67f22d40-f07f-4ee3-bbbd-5f1ed9d27e69_4,67f22d40-f07f-4ee3-bbbd-5f1ed9d27e69_26,67f22d40-f07f-4ee3-bbbd-5f1ed9d27e69_31,67f22d40-f07f-4ee3-bbbd-5f1ed9d27e69_48,...,67f22d40-f07f-4ee3-bbbd-5f1ed9d27e69_36,67f22d40-f07f-4ee3-bbbd-5f1ed9d27e69_41,67f22d40-f07f-4ee3-bbbd-5f1ed9d27e69_47,67f22d40-f07f-4ee3-bbbd-5f1ed9d27e69_44,67f22d40-f07f-4ee3-bbbd-5f1ed9d27e69_52,67f22d40-f07f-4ee3-bbbd-5f1ed9d27e69_42,67f22d40-f07f-4ee3-bbbd-5f1ed9d27e69_45,67f22d40-f07f-4ee3-bbbd-5f1ed9d27e69_49,67f22d40-f07f-4ee3-bbbd-5f1ed9d27e69_53,67f22d40-f07f-4ee3-bbbd-5f1ed9d27e69_46
f1_score_weighted,[0.7760223015120782],[0.9119113009643813],[0.8362720902551278],[0.9078537631686503],[0.8974606813595954],[0.8352528881918335],[0.9043728516821864],[0.905493134869961],[0.9113927869010032],[0.905287774344588],...,[0.8352528881918335],[0.908159307412182],[0.9001627778808211],[0.8352528881918335],[0.9162798364050319],[0.8352528881918335],[0.8906110015855894],[0.9039432607937848],[0.9120779881611449],[0.8352528881918335]
f1_score_macro,[0.5980892276431388],[0.7706487553528701],[0.47420454588426786],[0.7600804286945273],[0.7170048751133455],[0.47032494017168064],[0.7373176595782777],[0.7455101320132393],[0.7672208563645179],[0.7551902369551963],...,[0.47032494017168064],[0.7624441152167286],[0.7254507574632599],[0.47032494017168064],[0.784382334393945],[0.47032494017168064],[0.6939973893373192],[0.7529315864262568],[0.7696419041373481],[0.47032494017168064]
recall_score_micro,[0.7289834714874299],[0.9151744965408922],[0.8883459196037965],[0.9112897463397548],[0.908133554832595],[0.8879513842675975],[0.9137784354628531],[0.9125644850457162],[0.9155992933925097],[0.9080424663448955],...,[0.8879513842675975],[0.9109255692276154],[0.9101061596728858],[0.8879513842675975],[0.9183914680475643],[0.8879513842675975],[0.9040060659417652],[0.9062518505889291],[0.9159635202405217],[0.8879513842675975]
average_precision_score_weighted,[0.910857673097496],[0.9554048299266625],[0.9523421235516931],[0.9529117026453244],[0.9477898966333494],[0.9476979843446051],[0.9553413560130618],[0.953060723410316],[0.9540241256872021],[0.9485826619962011],...,[0.9311081985460802],[0.9513522625599348],[0.9492803569983689],[0.9361292656644511],[0.9559017800742002],[0.9412642154209729],[0.9405926914919159],[0.947997497108915],[0.9553261240736804],[0.9448100675971016]
recall_score_weighted,[0.7289834714874299],[0.9151744965408922],[0.8883459196037965],[0.9112897463397548],[0.908133554832595],[0.8879513842675975],[0.9137784354628531],[0.9125644850457162],[0.9155992933925097],[0.9080424663448955],...,[0.8879513842675975],[0.9109255692276154],[0.9101061596728858],[0.8879513842675975],[0.9183914680475643],[0.8879513842675975],[0.9040060659417652],[0.9062518505889291],[0.9159635202405217],[0.8879513842675975]
AUC_macro,[0.8207893149404871],[0.9484114349970835],[0.9406902573159694],[0.9444719738797162],[0.9319784930789078],[0.9319106536353434],[0.9460487900685927],[0.9423687255094976],[0.9431867991954538],[0.9385175061516894],...,[0.9084258513060601],[0.9428741536826664],[0.9346786329538338],[0.9113780285067824],[0.9478522566312786],[0.916847593406835],[0.9212953114986505],[0.936497618237917],[0.9470494488586193],[0.9266061985687474]
recall_score_macro,[0.7190151784031033],[0.751980578448754],[0.5018818622956122],[0.7421312817841198],[0.6810937257775045],[0.5],[0.6981183890554866],[0.7135455574196131],[0.7445583244850159],[0.7411374669416465],...,[0.5],[0.7475881531597368],[0.6879257972134942],[0.5],[0.7706796917049888],[0.5],[0.6600647538471945],[0.7413647217871304],[0.7481562716843362],[0.5]
average_precision_score_macro,[0.7050112642664348],[0.8239314075924975],[0.815029811917741],[0.8146659371640729],[0.8019138105013123],[0.7986723711881619],[0.8251475980300272],[0

### Retrieve the Best Model

In [21]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

Downloaded azureml/67f22d40-f07f-4ee3-bbbd-5f1ed9d27e69/model_data, 1 files out of an estimated total of 1


In [22]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, is_cross_validation=True, working_dir='/mnt/batch/tasks/shared/LS_root/mounts/clusters/ops-compute/code')),
                                                 ('prefittedsoftvotingclassifier',
                                                  PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), est...timators=10, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), reg_alpha=2.1875, reg_lambda=0.10416666666666667, subsample=0.5, tree_method='hist'))]))], flatten_transform=False, weights=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]))],
                                       'verbose': False},
                             y_transformer={},
                             y_transformer_name='LabelEncoder')

In [23]:
best_model.steps

[('datatransformer',
  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, task='classification')),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=numpy.array([0, 1]), estimators=[('31', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bylevel=0.6, colsample_bytree=1, eta=0.001, gamma=0, max_depth=6, max_leaves=15, n_estimators=800, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=0, reg_lambda=2.5, subsample=1, tree_method='auto'))], verbose=False)), ('28', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=Tr

## Publish and run from REST endpoint

Run the following code to publish the pipeline to your workspace. In your workspace in the portal, you can see metadata for the pipeline including run history and durations. You can also run the pipeline manually from the portal.

Additionally, publishing the pipeline enables a REST endpoint to rerun the pipeline from any HTTP library on any platform.


In [24]:
# Ensure that the workspace is once again available
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

udacity_ws
udacity_rg
eastus
051560d6-9344-4907-a9c5-057add5cf030


In [ ]:
experiment_name = 'bm_job'
project_folder = './pipeline-bike-project'

experiment = Experiment(ws, experiment_name)

In [36]:
from azureml.pipeline.core import PipelineRun

run_id = "6277ee6c-48bc-4532-adc2-e95fd790b9c8"
pipeline_run = PipelineRun(experiment, run_id)
pipeline_run

Experiment,Id,Type,Status,Details Page,Docs Page
bm_job,6277ee6c-48bc-4532-adc2-e95fd790b9c8,azureml.PipelineRun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [37]:
published_pipeline = pipeline_run.publish_pipeline(name="Bikesharing Train", description="Training bikesharing pipeline", version="1.0")
published_pipeline

Name,Id,Status,Endpoint
Bikesharing Train,397a04b6-ce30-4cf9-b1c9-000db4872fad,Active,REST Endpoint


Authenticate once again, to retrieve the `auth_header` so that the endpoint can be used

In [39]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

Get the REST url from the endpoint property of the published pipeline object. You can also find the REST url in your workspace in the portal. Build an HTTP POST request to the endpoint, specifying your authentication header. Additionally, add a JSON payload object with the experiment name and the batch size parameter. As a reminder, the process_count_per_node is passed through to ParallelRunStep because you defined it is defined as a PipelineParameter object in the step configuration.

Make the request to trigger the run. Access the Id key from the response dict to get the value of the run id.


In [40]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-bike-rest-endpoint"}
                        )

In [41]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Submitted pipeline run:  1c263bae-0107-405d-b33b-b8b0d3065aa4


Use the run id to monitor the status of the new run. This will take another 10-15 min to run and will look similar to the previous pipeline run, so if you don't need to see another pipeline run, you can skip watching the full output.

In [42]:
from azureml.pipeline.core.run import PipelineRun

published_pipeline_run = PipelineRun(ws.experiments["pipeline-bike-rest-endpoint"], run_id)
published_pipeline_run.get_details()

{'runId': '1c263bae-0107-405d-b33b-b8b0d3065aa4',
 'status': 'Completed',
 'startTimeUtc': '2025-06-22T14:18:56.095Z',
 'endTimeUtc': '2025-06-22T14:18:58.056699Z',
 'services': {},
 'properties': {'azureml.runsource': 'azureml.PipelineRun',
  'runSource': 'Unavailable',
  'runType': 'HTTP',
  'azureml.parameters': '{}',
  'azureml.continue_on_step_failure': 'False',
  'azureml.continue_on_failed_optional_input': 'True',
  'azureml.pipelineid': '397a04b6-ce30-4cf9-b1c9-000db4872fad',
  'azureml.pipelineComponent': 'pipelinerun',
  'azureml.pipelines.stages': '{"Initialization":null,"Execution":{"StartTime":"2025-06-22T14:18:56.4758545+00:00","EndTime":"2025-06-22T14:18:57.9558592+00:00","Status":"Finished"}}'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'logs/azureml/executionlogs.txt': 'https://udacityws6195221394.blob.core.windows.net/azureml/ExperimentRun/dcid.1c263bae-0107-405d-b33b-b8b0d3065aa4/logs/azureml/executionlogs.txt?sv=2019-07-07&sr=b&sig=GPvASH2Y0ZJmzDE8E